In [1]:
#default_exp haystack_search

In [2]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch
from sklearn import metrics
from nltk import tokenize
from operator import itemgetter

from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.database.memory import InMemoryDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from pytorch_hackathon import rss_feeds

import seaborn as sns

08/16/2020 13:01:42 - INFO - transformers.file_utils -   PyTorch version 1.5.0+cu101 available.
08/16/2020 13:01:43 - INFO - transformers.file_utils -   TensorFlow version 2.1.0 available.


In [3]:
pd.set_option('max_colwidth', 100)

In [4]:
cm = sns.light_palette("green", as_cmap=True)

In [5]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [6]:
!ls data

feeds.txt  topics.txt  zsl_feed_results.csv


In [7]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [8]:
feed_df = rss_feeds.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:07<00:00,  2.11it/s]
/home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py:63: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 63 of the file /home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [9]:
use_gpu = torch.cuda.is_available()

In [10]:
pretty_print = pprint.PrettyPrinter(indent=2).pprint

In [11]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,published_parsed,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail
0,Hybrid Dynamic-static Context-aware Attention Network for Action Assessment in Long Videos,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/hybrid-dyna...",https://paperswithcode.com/paper/hybrid-dynamic-static-context-aware-attention,"However, most existing works focus only on video dynamic information (i. e., motion information)...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/hybrid-dynamic-static-context-aware-attention,False,"[{'term': 'Action quality assessment', 'scheme': None, 'label': None}]","However, most existing works focus only on video dynamic information (i. e., motion information)...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Predicting Visual Overlap of Images Through Interpretable Non-Metric Box Embeddings,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/predicting-...",https://paperswithcode.com/paper/predicting-visual-overlap-of-images-through,"Even when this is a known scene, the answer typically requires an expensive search across scale ...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/predicting-visual-overlap-of-images-through,False,NaN,"Even when this is a known scene, the answer typically requires an expensive search across scale ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Statistical Evaluation of Anomaly Detectors for Sequences,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/statistical...",https://paperswithcode.com/paper/statistical-evaluation-of-anomaly-detectors,"Although precision and recall are standard performance measures for anomaly detection, their sta...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/statistical-evaluation-of-anomaly-detectors,False,"[{'term': 'Anomaly detection', 'scheme': None, 'label': None}]","Although precision and recall are standard performance measures for anomaly detection, their sta...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,On failures of RGB cameras and their effects in autonomous driving applications,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/on-failures...",https://paperswithcode.com/paper/on-failures-of-rgb-cameras-and-their-effects,RGB cameras are arguably one of the most relevant sensors for autonomous driving applications. <...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...",https://paperswithcode.com/paper/on-failures-of-rgb-cameras-and-their-effects,False,"[{'term': 'Autonomous driving', 'scheme': None, 'label': None}]",RGB cameras are arguably one of the most relevant sensors for autonomous driving applications. C...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Contextual Diversity for Active Learning,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/contextual-...",https://paperswithcode.com/paper/contextual-diversity-for-active-learning,Contextual Diversity (CD) hinges on a crucial observation that the probability vector predicted ...,"{'

In [12]:
#export



class Searcher:
    
    def __init__(
        self,
        model_name,
        text_col,
        use_gpu,
        max_document_length=256,
        quantize_model=True,
        document_store_cls=InMemoryDocumentStore
    ):
        self.text_col = text_col
        self.embedding_col = text_col + '_emb'
        self.max_document_length = max_document_length
        self.model_name = model_name
        self.document_store = document_store_cls(
            embedding_field=self.embedding_col,
        )
        self.retriever = self._setup_retriever(use_gpu, quantize_model)

    def _setup_retriever(self, use_gpu, quantize_model):
        retriever = EmbeddingRetriever(
            document_store=self.document_store,
            embedding_model=self.model_name,
            use_gpu=use_gpu)
        if not use_gpu and quantize_model:
            self.set_quantized_model(retriever)
            
        return retriever

    def add_texts(
        self,
        df
    ):
        truncated_texts = [
            ' '.join(tokenize.wordpunct_tokenize(text)[:self.max_document_length])
            for text in df[self.text_col] 
        ]
        article_embeddings = self.retriever.embed_queries(
            texts=truncated_texts
        )

        df[self.embedding_col] = article_embeddings
        self.document_store.write_documents(df.to_dict(orient='records'))
    
    @classmethod
    def set_quantized_model(cls, retriever):
        quantized_model = torch.quantization.quantize_dynamic(
            retriever.embedding_model.model,
            {torch.nn.Linear}, dtype=torch.qint8
        )
        retriever.embedding_model.model = quantized_model
        
    @classmethod 
    def sigmoid(cls, x):
        return 1 / (1 + np.exp(-x))
    
    @classmethod
    def doc_to_dict(cls, doc):
        d = {}
        d['text'] = doc.text
        d['title'] = doc.meta['title']
        d['score'] = doc.query_score
        return d

    def get_topic_score_df(self, raw_results, topic_strings):
        topic_query_strings = [
            'text is about {}'.format(topic)
            for topic in topic_strings
        ]

        results = [
            self.doc_to_dict(doc)
            for doc in raw_results 
        ]
        result_embeddings = np.array([
            doc.meta['text_emb']
            for doc in raw_results
        ]).astype('float32')
        topic_query_embeddings = np.array(self.retriever.embed_passages(
            list(topic_strings)
        )).astype('float32')

        scores_df = pd.DataFrame({})
        scores_df['title'] = list(map(itemgetter('title'), results))
        scores_df['text'] = list(map(itemgetter('text'), results))

        scores = pd.DataFrame(metrics.pairwise.cosine_similarity(
            result_embeddings,
            topic_query_embeddings
        ))
        scores.columns = topic_strings

        scores_df = pd.concat(
            [scores_df, self.sigmoid(scores)],
            axis=1
        )
        return scores_df

In [13]:
model_name = "deepset/sentence_bert"

In [14]:
searcher = Searcher(
    model_name,
    'text',
    use_gpu=use_gpu
)

08/16/2020 13:01:52 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
08/16/2020 13:01:52 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/16/2020 13:01:52 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
08/16/2020 13:01:53 - INFO - transformers.modeling_utils -   loading weights file https://cdn.huggingface.co/deepset/sentence_bert/pytorch_model.bin from cache at /home/kuba/.cache/torch/transformers/fa9d12cb00cd5a31f5a5367f58d242199473a6deb02c51380681ade7bf33c713.4948a08b5d844db1ecda79f6e7f47643f0175f2c030d48ce8b3beee3c6bd6012
08/16/2020 13:01:54 - INFO - transformers.modeling_utils -   All model checkpoint weights were used when initializing BertModel.

08/16/2020 13:01:54 - INFO - transformers.modeling_utils -   All the weights of BertModel were initialized from the model checkpoint at deepset/sentence_be

In [15]:
searcher.add_texts(feed_df)

08/16/2020 13:02:02 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/16/2020 13:02:02 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 57-0
Clear Text: 
 	text: Comments
Tokenized: 
 	tokens: ['comments']
 	offsets: [0]
 	start_of_word: [True]
Features: 
 	input_ids: [101, 7928, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Inferencing Samples: 100%|██████████| 74/74 [00:10<00:00,  7.08 Batches/s]


In [16]:
article_texts = feed_df['text']

In [17]:
topic_strings = pd.read_table('data/topics.txt', header=None).iloc[:,0].values

In [18]:
print('\n'.join(topic_strings))

deep learning
natural language processing
computer vision
statistics
implementation
visualization
industry
software engineering
reddit question
arxiv
cloud computing
deployment
competitions
business
business intelligence


In [19]:
topic_query_strings = [
    'text is about {}'.format(topic)
    for topic in topic_strings
]

In [20]:
raw_results = searcher.retriever.retrieve(
    topic_query_strings[1]
)

08/16/2020 13:02:12 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/16/2020 13:02:12 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 0-0
Clear Text: 
 	text: text is about natural language processing
Tokenized: 
 	tokens: ['text', 'is', 'about', 'natural', 'language', 'processing']
 	offsets: [0, 5, 8, 14, 22, 31]
 	start_of_word: [True, True, True, True, True, True]
Features: 
 	input_ids: [101, 3793, 2003, 2055, 3019, 2653, 6364, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 31.63 Batches/s]


In [21]:
scores_df = searcher.get_topic_score_df( raw_results, topic_strings)

08/16/2020 13:02:12 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
08/16/2020 13:02:12 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: 9-0
Clear Text: 
 	text: arxiv
Tokenized: 
 	tokens: ['ar', '##xi', '##v']
 	offsets: [0, 2, 4]
 	start_of_word: [True, False, False]
Features: 
 	input_ids: [101, 12098, 9048, 2615, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00,  7.69 Batches/s]
08/16/2020 13:02:13 - INFO - numexpr.utils -   Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
08/16/2020 13:02:13 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.


In [22]:
scores_df.style.background_gradient(cmap=cm)

In [23]:
[doc.text for doc in raw_results]

['Pretrained Transformer models have emerged as state-of-the-art approaches that learn contextual information from the text to improve the performance of several NLP tasks. Code: https://github.com/mulangonando/Impact-of-KG-Context-on-ED',
 'Converting tokens of text into features and applying machine learning and deep learning model on it.Continue reading on Towards AI\u200a—\u200aMultidisciplinary Science Journal »',
 'NLP is a great tool to analyze text data and perform an amazing task when combined with machine learning and deep learning. So, let’s look…Continue reading on Towards AI\u200a—\u200aMultidisciplinary Science Journal »',
 'As a legal platform, Doctrine aggregates a lot of legal data with the intent of making them accessible, understandable and usable. The Machine Learning Engineers’ day-to-day material is mostly text: court decisions, legislation, legal commentaries, user queries, etc. All of our content is natural language, which we process in a number of ways: bag-of-

In [24]:
scores_df

,title,text,deep learning,natural language processing,computer vision,statistics,implementation,visualization,industry,software engineering,reddit question,arxiv,cloud computing,deployment,competitions,business,business intelligence
0,Evaluating the Impact of Knowledge Graph Context on Entity Disambiguation Models,Pretrained Transformer models have emerged as state-of-the-art approaches that learn contextual ...,0.613387,0.597722,0.531973,0.501481,0.593792,0.559076,0.545501,0.592912,0.543777,0.490492,0.511187,0.537554,0.530916,0.494104,0.496908
1,Feature Engineering with NLP,Converting tokens of text into features and applying machine learning and deep learning model on...,0.597072,0.590448,0.549067,0.511888,0.567869,0.536750,0.531344,0.617110,0.547273,0.493482,0.531765,0.509824,0.536120,0.519174,0.530965
2,Introduction to Natural Language Processing,NLP is a great tool to analyze text data and perform an amazing task when combined with machine ...,0.606703,0.586747,0.556306,0.513555,0.553962,0.538722,0.533227,0.608819,0.549520,0.514010,0.531522,0.506409,0.531768,0.515964,0.540105
3,A single legal text representation at Doctrine: the legal camemBERT,"As a legal platform, Doctrine aggregates a lot of legal data with the intent of making them acce...",0.570231,0.586314,0.521751,0.499045,0.548911,0.511261,0.517473,0.595577,0.505991,0.492187,0.509414,0.505175,0.512034,0.511341,0.525712
4,A Large-Scale Chinese Short-Text Conversation Dataset,The cleaned dataset and the pre-training models will facilitate the research of short-text conve...,0.621184,0.586691,0.545824,0.541583,0.584535,0.550647,0.527896,0.600209,0.548811,0.484411,0.517256,0.519267,0.528229,0.519199,0.530539
5,"[Q] Data scientist here, working on gathering a corpus of academic papers focusing on ""Cognitive...",Hello. I want to collect as many as papers as I can that will fall into this category. The main ...,0.555300,0.578011,0.498356,0.489413,0.526189,0.513210,0.489906,0.544217,0.557324,0.480271,0.496461,0.485666,0.530694,0.496379,0.504614
6,KR-BERT: A Small-Scale Korean-Specific Language Model,"Since the appearance of BERT, recent works including XLNet and RoBERTa utilize sentence embeddin...",0.574174,0.594593,0.522638,0.532562,0.583562,0.552562,0.547111,0.580898,0.551625,0.498453,0.515522,0.564181,0.550184,0.508991,0.513304
7,Dialogue State Induction Using Neural Latent Variable Models,Dialogue state modules are a useful component in a task-oriented dialogue system. Code: https://...,0.609547,0.595336,0.546996,0.492073,0.601822,0.568787,0.546473,0.602195,0.570310,0.499499,0.532563,0.543948,0.563158,0.529331,0.538693
8,Inter-Image Communication for Weakly Supervised Localization,We learn a feature center for each category and realize the global feature consistency by forcin...,0.593860,0.596002,0.541500,0.521783,0.593086,0.556799,0.535707,0.601382,0.539489,0.486664,0.553956,0.532525,0.545695,0.532982,0.542407
9,TextRay: Contour-based Geometric Modeling for Arbitrary-shaped Scene Text Detection,Arbitrary-shaped text detection is a challenging task due to the complex geometric layouts of te...,0.573135,0.569583,0.536931,0.505102,0.543021,0.542183,0.517943,0.571432,0.553461,0.498335,0.522298,0.509426,0.536340,0.498024,0.506026
